# Compare modeled and remotely-sensed surface mass balance

Requires mass balance model outputs at each site from [PyGEM](https://github.com/PyGEM-Community/PyGEM) (Rounce et al., 2023), which can be downloaded form the Carnegie Mellon data repository.

The files downloaded for this work were:
- Monthly surface mass balance along glacier centerlines for 2000–2022: "binned", downloaded from [global_ERA5_2000_2022](https://cmu.app.box.com/s/rzk8aeasg40dd3p0xr3yngkc5c0m8kxt/folder/251139952066)
- Calibrated model parameters (degree-day factors of snow and temperature biases): "{RGI ID}_modelprms_dict.json" files downloaded from [pygem_datasets > Calibration](https://cmu.app.box.com/s/p8aiby5s9f3n6ycgmhknbgo4htk3pn9j/folder/298954564072)

All files were placed in a folder called "Rounce_et_al_2023", defined with the `model_path` variable below. 

In [ ]:
import os
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import median_abs_deviation as MAD
from tqdm import tqdm
import glob
import geopandas as gpd
import json
import seaborn as sns
import sys

In [ ]:
# Define paths for inputs and outputs
scm_path = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/'
model_path = os.path.join(scm_path, 'Rounce_et_al_2023')
out_path = os.path.join(scm_path, 'dataset', 'model_comparison')

# import utility functions
code_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/glacier-snow-cover-analysis'
figures_path = os.path.join(code_path, 'figures')
sys.path.append(os.path.join(code_path, 'scripts'))
import utils as f

# Load glacier boundaries for RGI IDs
aois_fn = os.path.join(scm_path, 'dataset', 'AOIs.gpkg')
aois = gpd.read_file(aois_fn)

## 1. Monthly snowline altitudes

### Remotely-sensed SLAs

In [ ]:
slas_obs_fn = os.path.join(out_path, 'monthly_SLAs_observed.nc')
if not os.path.exists(slas_obs_fn):
    # Initialize a list to store DataFrames
    slas_obs_list = []
    
    # sample SLA observations +/- 1 week of the first of each month
    tdelta = np.timedelta64(1, 'W')  

    # Iterate over RGI IDs
    for rgi_id in tqdm(sorted(aois['RGIId'].drop_duplicates().values)):
        scs_fn = os.path.join(scm_path, 'study-sites', rgi_id, f'{rgi_id}_classifications.zarr')
        scs = f.load_snow_cover_stats(scs_fn)
        scs = scs.assign_coords({'Year': scs['time'].dt.isocalendar().year})
        scs = scs.assign_coords({'Month': scs['time'].dt.month})
        scs = scs.assign_coords({'Day': scs['time'].dt.day})
        
        # Grab monthly snowline altitude
        dates = []
        slas = []
        nobs = []
        for year, month in np.unique(np.array(list(zip(scs['Year'].values, scs['Month'].values))), axis=0):
            target_time = np.datetime64(f'{year}-0{month}-01') if month < 10 else np.datetime64(f'{year}-{month}-01')
            scs_near = scs.sel(time=(scs['time'] >= target_time - tdelta) & 
                                    (scs['time'] <= target_time + tdelta))
            scs_near = scs_near.dropna(dim='time')
            if len(scs_near['SLA'].values) > 0:
                dates.append(target_time)
                slas.append(float(scs_near['SLA'].mean().values))
                nobs.append(len(scs_near['SLA'].values))
        
        scs_monthly = pd.DataFrame({'RGIId': [rgi_id]*len(dates), 
                                    'Date': dates,
                                    'SLA_obs': slas,
                                    'num_obs': nobs})
        slas_obs_list.append(scs_monthly)
    
    # Combine all DataFrames
    slas_obs = pd.concat(slas_obs_list)
    
    # Pivot
    slas_obs_pivot = slas_obs.pivot(index='Date', columns='RGIId', values='SLA_obs')
    slas_obs_pivot = slas_obs_pivot.sort_index()
    
    # Convert to xarray Dataset
    slas_obs_xr = xr.Dataset(
        {"SLA_obs": (['time', 'RGIId'], slas_obs_pivot.values)},
        coords={"time": slas_obs_pivot.index.values,
                "RGIId": slas_obs_pivot.columns.values}
    )

    # set attributes
    slas_obs_xr['SLA_obs'].attrs['long_name'] = 'observed snowline altitude'
    slas_obs_xr['SLA_obs'].attrs['units'] = 'meters above sea level'
    
    # Save to NetCDF file
    slas_obs_xr.to_netcdf(slas_obs_fn)
    print('Remotely-sensed monthly SLAs saved to file:', slas_obs_fn)

else:
    # Load from file
    slas_obs_xr = xr.load_dataset(slas_obs_fn)
    print('Remotely-sensed monthly SLAs loaded from file.')

slas_obs_xr


### Modeled SLAs and SMB at observed SLAs

In [ ]:
# Check if file already exists
slas_mod_fn = os.path.join(out_path, 'monthly_SLAs_modeled.nc')
if not os.path.exists(slas_mod_fn):
    
    # Initialize a list to store DataFrames
    slas_mod_list = []
    
    # Iterate over sites
    for rgi_id in tqdm(aois['RGIId'].drop_duplicates().values):
        # Load modeled monthly SMB
        smb_fn = glob.glob(os.path.join(model_path, 'glac_SMB_binned', f"{rgi_id.split('RGI60-0')[1]}*.nc"))[0]
        smb = xr.open_dataset(smb_fn)
        # calculate cumulative SMB
        def water_year(date):
            if date.month >= 10:
                return date.year
            else:
                return date.year - 1
        smb = smb.assign_coords({'water_year': (['time'], [water_year(t) for t in smb.time.values])})
        smb['bin_massbalclim_monthly_cumsum'] = smb['bin_massbalclim_monthly'].groupby('water_year').cumsum()
        smb['time'] = smb.time.values.astype('datetime64[D]')
        h = smb['bin_surface_h_initial'].data.ravel()
        
        # Interpolate modeled SLA as where SMB = 0 and SMB at the observed SLA
        slas = np.nan * np.zeros(len(smb.time.data))
        smb_at_slas = np.nan * np.zeros(len(smb.time.data))
        for j, t in enumerate(smb.time.data):
            smb_time = smb.sel(time=t)['bin_massbalclim_monthly_cumsum'].data[0]
            # when SMB <= 0 everywhere, set SLA to maximum glacier elevation
            if np.all(smb_time <= 0):
                slas[j] = np.max(h)
            # when SMB >= 0 everywhere, set SLA to minimum glacier elevation
            elif np.all(smb_time >= 0):
                slas[j] = np.min(h)
            # otherwise, linearly interpolate SLA
            else:
                sorted_indices = np.argsort(h)
                slas[j] = np.interp(0, smb_time[sorted_indices], h[sorted_indices])
            # interpolate the modeled SMB at the observed SLA
            sla_obs = slas_obs.loc[(slas_obs['RGIId']==rgi_id) & (slas_obs['Date']==t), 'SLA_obs']
            if len(sla_obs) > 0:
                smb_at_slas[j] = np.interp(sla_obs.values[0], h, smb_time)

        # Save results in dataframe
        df = pd.DataFrame({'RGIId': [rgi_id]*len(smb.time.data),
                            'Date': smb.time.data,
                            'SLA_mod': slas,
                            'SMB_at_SLA_obs': smb_at_slas})
        # concatenate to dataframe list
        slas_mod_list.append(df)
        
    # Combine all DataFrames
    slas_mod = pd.concat(slas_mod_list)
    
    # Create xarray Dataset
    slas_mod_pivot = slas_mod.pivot(index='Date', columns='RGIId', values=['SLA_mod', 'SMB_at_SLA_obs'])
    slas_mod_pivot = slas_mod_pivot.sort_index()
    
    # Convert to xarray Dataset
    slas_mod_xr = xr.Dataset(
        {"SLA_mod": (['time', 'RGIId'], slas_mod_pivot['SLA_mod'].values),
         "SMB_at_SLA_obs":(['time', 'RGIId'], slas_mod_pivot['SMB_at_SLA_obs'].values)},
        coords={"time": slas_mod_pivot.index.values,
                "RGIId": slas_mod_pivot.columns.levels[1].values}
    )

    # assign attributes
    slas_mod_xr['SLA_mod'].attrs['long_name'] = 'modeled snowline altitude'
    slas_mod_xr['SLA_mod'].attrs['units'] = 'meters above sea level'
    slas_mod_xr['SMB_at_SLA_obs'].attrs['long_name'] = 'modeled surface mass balance at observed snowline altitude'
    slas_mod_xr['SMB_at_SLA_obs'].attrs['units'] = 'meters water equivalent'

    # Save to NetCDF file
    slas_mod_xr.to_netcdf(slas_mod_fn)
    print('Modeled monthly SLAs and snowline SMB saved to file:', slas_mod_fn)
    
else:
    # Load from file
    slas_mod_xr = xr.load_dataset(slas_mod_fn)
    print('Modeled monthly SLAs loaded from file.')

slas_mod_xr

### Merge

In [ ]:
# Define output file
slas_merged_fn = os.path.join(out_path, 'monthly_SLAs_observed_modeled.nc')
if not os.path.exists(slas_merged_fn):

    # Merge modeled and remotely-sensed SLAs and modeled SMB at observed snowline
    slas_merged = xr.merge([slas_obs_xr, slas_mod_xr])
    
    # Remove 2000-2012 (no observed values) and 2023 (no modeled values)
    slas_merged.sel(time=slice("2013-01-01","2023-01-01"))
    
    # Remove observations outside May - November (no observed values)
    def filter_month_range(month):
        return (month >= 5) & (month <= 10)
    slas_merged = slas_merged.sel(time=filter_month_range(slas_merged['time.month']))
    
    # Save results
    slas_merged.to_netcdf(slas_merged_fn)
    print('Merged monthly SLAs saved to file:', slas_merged_fn)

else:
    slas_merged = xr.load_dataset(slas_merged_fn)
    print('Merged monthly SLAs loaded from file.')

slas_diff = slas_merged['SLA_mod'] - slas_merged['SLA_obs']

# Plot
fig, ax = plt.subplots(figsize=(6,5))
ax.hist(slas_diff.values.ravel(), bins=50)
ax.set_xlabel('SLA$_{mod}$ - SLA$_{obs}$ [m]')
ax.set_ylabel('Counts')
plt.show()

print('\nDifference stats:')
print(f'Mean diff = {np.nanmean(slas_diff.values)} m')
print(f'Std. diff = {np.nanstd(slas_diff.values)} m')
print(f'Median diff = {np.nanmedian(slas_diff.values)} m')
print(f'MAD diff = {MAD(slas_diff.values.ravel(), nan_policy="omit")} m')

## 2. ELAs

In [ ]:
slas_elas_merged_fn = os.path.join(out_path, 'monthly_SLAs_annual_ELAs_observed_modeled.nc')

if not os.path.exists(slas_elas_merged_fn):
    # Make a copy of the SLAs
    slas_elas_merged = slas_merged.copy()
    
    # Identify maximum annual observed SLAs
    slas_elas_merged['ELA_obs'] = slas_elas_merged['SLA_obs'].groupby(['time.year']).max()
    # Set ELAs before 2016 to NaN
    slas_elas_merged['ELA_obs'] = xr.where(slas_elas_merged['ELA_obs'].year < 2016, np.nan, slas_elas_merged['ELA_obs'])
    
    # Identify maximum annual modeled SLAs 
    slas_elas_merged['ELA_mod'] = slas_elas_merged['SLA_mod'].groupby(['time.year']).max()
    
    # Set attributes
    slas_elas_merged['ELA_obs'].attrs['units'] = 'meters above sea level'
    slas_elas_merged['ELA_obs'].attrs['long_name'] = 'observed equilibrium line altitude'
    slas_elas_merged['ELA_mod'].attrs['units'] = 'meters above sea level'
    slas_elas_merged['ELA_mod'].attrs['long_name'] = 'modeled equilibrium line altitude'
    slas_elas_merged.attrs['title'] = 'Observed and Modeled Snowline and Equilibrium Line Altitudes'
    slas_elas_merged.attrs['description'] = (
        'Time series of observed and modeled snowline altitudes (SLAs) and equilibrium line altitudes (ELAs). '
        'Observed values were derived using the glacier-snow-cover-mapping pipeline, while modeled values '
        'come from surface mass balance estimates produced by PyGEM (Rounce et al., 2023).'
    )
    slas_elas_merged.attrs['institution'] = 'Boise State University'
    slas_elas_merged.attrs['references'] = 'doi:10.1029/2025GL115523'
    slas_elas_merged.attrs['date_modified'] = '2025-06-07'
    slas_elas_merged.attrs['time_coverage_start'] = '2013-05-01'
    slas_elas_merged.attrs['time_coverage_end'] = '2023-10-31'
    slas_elas_merged.attrs['horizontal_CRS'] = 'WGS84 (EPSG:4326)'
    slas_elas_merged.attrs['vertical_CRS'] = 'EGM96 geoid (EPSG:5773)'
    slas_elas_merged.attrs['model_source'] = 'PyGEMv1.0.1'
    slas_elas_merged.attrs['model_institution'] = 'Carnegie Mellon University, Pittsburgh PA'
    slas_elas_merged.attrs['model_history'] = 'Created by David Rounce (drounce@cmu.edu) on 2025-04-11'
    slas_elas_merged.attrs['model_references'] = 'doi:10.1126/science.abo1324'


    # Save to NetCDF file
    slas_elas_merged.to_netcdf(slas_elas_merged_fn)
    print('Merged SLAs and ELAs saved to file:', slas_elas_merged_fn)

else:
    slas_elas_merged = xr.load_dataset(slas_elas_merged_fn)
    print('Merged SLAs and ELAs loaded from file.')

slas_elas_merged

In [ ]:
slas_elas_merged['ELA_mod-obs'] = slas_elas_merged['ELA_mod'] - slas_elas_merged['ELA_obs']
plt.hist(slas_elas_merged['ELA_mod-obs'].values.ravel(), bins=50)
plt.show()

print('\nDifference stats:')
print(f"Mean diff = {np.nanmean(slas_elas_merged['ELA_mod-obs'].values)} m")
print(f"Std. diff = {np.nanstd(slas_elas_merged['ELA_mod-obs'].values)} m")
print(f"Median diff = {np.nanmedian(slas_elas_merged['ELA_mod-obs'].values)} m")
print(f"MAD diff = {MAD(slas_elas_merged['ELA_mod-obs'].values.ravel(), nan_policy='omit')} m")

## 3. Assess agreement with new PyGEM runs at the USGS Benchmark Glaciers

For remotely-sensed and modeled snowline time series, bin estimates by week of year, characterize the timing of snowline rise and fall and maximum snowline altitude. Select the PyGEM run that agrees best.  

In [ ]:
import warnings
warnings.filterwarnings('ignore')

plot_results = False  # Set to True to generate plots

# Define paths
summary_fn = os.path.join(out_path, 'PyGEM_comparison_summary.nc')
pygem_new_path = os.path.join(scm_path, 'Brandon_new_PyGEM_runs')

if not os.path.exists(summary_fn):
    # Glacier identifiers and names
    rgi_ids = [x for x in sorted(os.listdir(pygem_new_path)) if os.path.isdir(os.path.join(pygem_new_path, x))]
    names = ['Gulkana', 'Wolverine', 'Lemon Creek', 'Sperry', 'South Cascade']
    print('RGI IDs for glaciers with PyGEM runs:', rgi_ids)

    # Initialize lists to collect structured outputs
    glacier_ids = []
    glacier_names = []
    original_params = []
    best_params = []
    rmse_orig_all = []
    rmse_best_all = []

    for i, rgi_id in enumerate(rgi_ids):
        name = names[i]
        print(name, rgi_id)

        # Get file names of model runs
        run_fns = sorted(glob.glob(os.path.join(pygem_new_path, rgi_id, '*.nc')))
        out_fn = os.path.join(out_path, f"PyGEM_comparison_RGI60-0{rgi_id}.nc")

        # Load or build model dataset
        if not os.path.exists(out_fn):
            # Load observed snow cover data
            scs_fn = os.path.join(scm_path, 'study-sites', f"RGI60-0{rgi_id}", f"RGI60-0{rgi_id}_classifications.zarr")
            scs = f.load_snow_cover_stats(scs_fn)

            # Iterate over model runs
            model_runs_list = []
            for fn in tqdm(run_fns):
                ds = xr.open_dataset(fn)
                ds['time'] = ds.indexes['time'].to_datetimeindex()
                params = json.loads(ds.model_parameters)
                run_ds = xr.Dataset({
                    'SLA_mod': ds['glac_snowline_monthly'],
                    'ELA_mod': ds['glac_ELA_annual'],
                    'kp': xr.DataArray(params['kp'], dims=()),
                    'tbias': xr.DataArray(params['tbias'], dims=()),
                    'ddfsnow': xr.DataArray(params['ddfsnow'], dims=()),
                })
                model_runs_list.append(run_ds)

            combined_ds = xr.concat(model_runs_list, dim='run')
            combined_ds = combined_ds.sel(time=slice('2013-01-01', None))
            combined_ds = combined_ds.sel(time=combined_ds['time.month'].isin([5, 6, 7, 8, 9, 10]))
            combined_ds = xr.merge([combined_ds.isel(glac=0), slas_obs_xr.sel(RGIId=f"RGI60-0{rgi_id}")])

            combined_ds.attrs.update({
                'title': 'PyGEM Simulations with Parameter Variations and Observed Snowline Altitudes at USGS Benchmark Glaciers',
                'description': 'Results from PyGEM simulations with varying model parameter values, combined with observed snowline altitudes at USGS Benchmark Glaciers.',
                'institution': 'Boise State University',
                'references': 'doi:10.1029/2025GL115523',
                'vertical_CRS': 'EGM96 geoid (EPSG:5773)',
                'date_modified': '2025-06-24',
                'time_coverage_start': '2013-05-01',
                'time_coverage_end': '2023-10-31'
            })
            combined_ds.to_netcdf(out_fn)
        else:
            combined_ds = xr.open_dataset(out_fn)

        # Load original parameters
        modelprms_fn = os.path.join(model_path, '..', 'Rounce_et_al_2023', 'modelprms', f"{rgi_id}-modelprms_dict.pkl")
        modelprms = pd.read_pickle(modelprms_fn)

        # Calculate RMSE and SLA difference
        diff = combined_ds['SLA_mod'] - combined_ds['SLA_obs']
        combined_ds['rmse'] = np.sqrt((diff**2).mean(dim='time'))
        combined_ds['SLA_mod-obs'] = diff

        # Best run based on lowest RMSE
        df_plot = combined_ds[['tbias', 'ddfsnow', 'kp', 'rmse', 'SLA_mod-obs']].to_dataframe().reset_index()
        df_plot = df_plot.dropna(subset=['rmse'])
        df_plot_best = df_plot.loc[df_plot['rmse'].idxmin()]

        # Find original run (closest param match)
        squared_diffs = sum((combined_ds[var] - modelprms['emulator'][var][0])**2 for var in ['kp', 'tbias', 'ddfsnow'])
        best_run_idx = squared_diffs.argmin(dim="run")
        combined_ds_original = combined_ds.sel(run=best_run_idx)
        combined_ds_best = combined_ds.sel(run=int(df_plot_best['run']))

        # Store structured summary
        glacier_ids.append(f"RGI60-0{rgi_id}")
        glacier_names.append(name)
        original_params.append([
            modelprms['emulator']['tbias'][0],
            modelprms['emulator']['ddfsnow'][0],
            modelprms['emulator']['kp'][0]
        ])
        best_params.append([
            df_plot_best['tbias'],
            df_plot_best['ddfsnow'],
            df_plot_best['kp']
        ])
        rmse_orig = float(np.sqrt(((combined_ds_original['SLA_mod'] - combined_ds_original['SLA_obs'])**2).mean().item()))
        rmse_best = float(np.sqrt(((combined_ds_best['SLA_mod'] - combined_ds_best['SLA_obs'])**2).mean().item()))
        rmse_orig_all.append(rmse_orig)
        rmse_best_all.append(rmse_best)

        # Optional: Plotting
        if plot_results:
            fig, ax = plt.subplots(3, 1, figsize=(10, 8), gridspec_kw=dict(height_ratios=[2,2,1]))
            sns.scatterplot(df_plot, x='tbias', y='ddfsnow', size='kp', hue='rmse', palette='viridis_r', sizes=(2,50), ax=ax[0])
            ax[0].plot(modelprms['emulator']['tbias'], modelprms['emulator']['ddfsnow'], 's', markersize=15, markeredgecolor='m', markerfacecolor='None', markeredgewidth=2, label='Original')
            ax[0].plot(df_plot_best['tbias'], df_plot_best['ddfsnow'], '*', markersize=15, markeredgecolor='m', markerfacecolor='None', markeredgewidth=2, label='Best')
            ax[0].legend()
            ax[0].set_title(f'{name} ({rgi_id})')
            ax[0].grid(True)
            ax[1].plot(combined_ds_original.time, combined_ds_original['SLA_mod'], '-', color='gray', label='Original')
            ax[1].plot(combined_ds_best.time, combined_ds_best['SLA_mod'], '-k', label='Best')
            ax[1].plot(scs['time'], scs['SLA'], '.m', markersize=5, label='Observed')
            ax[1].legend()
            ax[1].set_ylabel('Snowline altitude [m]')
            ax[2].plot(combined_ds_original.time, combined_ds_original['SLA_mod-obs'], '.', color='gray', label='Original')
            ax[2].plot(combined_ds_best.time, combined_ds_best['SLA_mod-obs'], '.', color='k', label='Best')
            ax[2].legend()
            ax[2].grid()
            ax[2].set_ylabel('Modeled - Observed\nSnowline Altitude [m]')
            fig.tight_layout()
            fig_fn = os.path.join(figures_path, f"{rgi_id}_PyGEM_comparison.png")
            fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
            print('Saved figure:', fig_fn)
            plt.close(fig)

    # Construct final dataset
    params = ['tbias', 'ddfsnow', 'kp']
    summary_ds = xr.Dataset(
        data_vars=dict(
            param_original=(['RGIId', 'parameter'], np.array(original_params)),
            param_best=(['RGIId', 'parameter'], np.array(best_params)),
            rmse_original=(['RGIId'], np.array(rmse_orig_all)),
            rmse_best=(['RGIId'], np.array(rmse_best_all)),
        ),
        coords=dict(
            RGIId=('RGIId', glacier_ids),
            parameter=('parameter', params),
            glacier_name=('RGIId', glacier_names),
        ),
        attrs=dict(
            title="PyGEM Parameter Comparison",
            description="Original and calibrated PyGEM parameters by glacier with associated RMSEs.",
            institution="Boise State University",
            references="doi:10.1029/2025GL115523",
            date_modified="2025-06-24",
            model_source='PyGEMv1.0.1',
            model_institution='Carnegie Mellon University, Pittsburgh PA',
            model_history='Created by David Rounce (drounce@cmu.edu) on 2025-04-11',
            model_references='doi:10.1126/science.abo1324'
        )
    )
    # add variable attributes
    summary_ds['param_original'].attrs.update({
        'long_name': 'Original PyGEM model parameter values',
        'description': 'Values from the default (uncalibrated) PyGEM model setup for each glacier'
    })
    summary_ds['param_best'].attrs.update({
        'long_name': 'Calibrated PyGEM model parameter values',
        'description': (
            'Best-fitting parameter values identified by selecting the PyGEM run that minimized the '
            'root mean square error (RMSE) between observed and modeled monthly snowline altitudes'
        )
    })
    summary_ds['rmse_original'].attrs.update({
        'long_name': 'RMSE for original PyGEM run',
        'units': 'meters',
        'description': (
            'Root mean square error (RMSE) between observed and modeled snowline altitudes for the PyGEM run '
            'using original (default) parameters'
        )
    })
    summary_ds['rmse_best'].attrs.update({
        'long_name': 'RMSE for calibrated PyGEM run',
        'units': 'meters',
        'description': (
            'Root mean square error (RMSE) between observed and modeled snowline altitudes for the PyGEM run '
            'with best-fit (calibrated) parameters'
        )
    })

    # Save to file
    summary_ds.to_netcdf(summary_fn)
    print('Results saved to file:', summary_fn)
    
else:
    # Load from file
    summary_ds = xr.open_dataset(summary_fn)
    print('Results loaded from file.')

summary_ds